In [1]:
import sys
sys.path.append("..") 

In [4]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import os
import torch.cuda
import albumentations as albu
import segmentation_models_pytorch as smp
from  segmentation_models_pytorch.utils.base import Metric
from segmentation_models_pytorch.base.modules import Activation
from collections import defaultdict
from Utils.dataset_utils import *

In [5]:
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [6]:
DATASET_2D_BASE_PATH=Path('G:\ML Project Datasets\Medical Image\MM-WH 2017 Dataset\Axials Version')
WIDTH=320
HEIGHT=320

In [7]:


def get_training_augmentation():
    train_transform = [

        albu.Resize(HEIGHT, WIDTH),
        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.2, p=1, border_mode=cv2.BORDER_CONSTANT),
        albu.RandomCrop(height=320, width=320, always_apply=True),
        albu.Blur(blur_limit=3, p=0.4),
        albu.GaussNoise(p=0.5),
        albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        albu.RandomBrightness(p=0.75)

    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    test_transform = [
        albu.Resize(512, 512)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [8]:
df_train=load_data(DATASET_2D_BASE_PATH/'train')[0:24]
df_val=load_data(DATASET_2D_BASE_PATH/'val')[0:8]
df_train

,images,masks
0,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
1,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
2,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
3,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
4,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
5,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
6,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
7,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
8,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...
9,G:\ML Project Datasets\Medical Image\MM-WH 201...,G:\ML Project Datasets\Medical Image\MM-WH 201...


In [9]:
class DCS(Metric):
    __name__ = 'DCS'

    def __init__(self, eps=0.00001, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
     
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        dice_numerator = 2 * torch.sum(y_pr * y_gt) + self.eps
        dice_denominator = torch.sum(y_pr) + torch.sum(y_gt) + self.eps
        dice_coefficient = dice_numerator / dice_denominator
        return dice_coefficient


g:\projects and work\mouse heart segmentation\venv\lib\site-packages\albumentations\augmentations\transforms.py:1802: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


NameError: name 'preprocessing_fn' is not defined

NameError: name 'train_dataset' is not defined

### Training

In [ ]:
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'
TRAIN_RUNS_PATH=r'G:\Projects and Work\Mouse Heart Segmentation\runs'
MODEL_NAME='Unet'
BATCH_SIZE=8
EPOCHS=10


In [ ]:
ENCODERS = ['se_resnext50_32x4d']

In [ ]:
for j in range(len(ENCODERS)):
    
    model = smp.Unet(
    encoder_name=ENCODERS[j], 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=1, 
    activation=ACTIVATION,
    )
    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODERS[j], ENCODER_WEIGHTS)
    
    WEIGHTS_PATH=os.path.join(TRAIN_RUNS_PATH,f'{MODEL_NAME}_{ENCODERS[j]}')
    if os.path.exists(WEIGHTS_PATH)==False:
        os.mkdir(WEIGHTS_PATH)
    else:
        print(f"Warning! Directory {WEIGHTS_PATH } already exists")
    
    train_dataset = Dataset(
    df_train['images'], 
    df_train['masks'], 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn)
    )

    valid_dataset = Dataset(
        df_val['images'], 
        df_val['masks'], 
        augmentation=get_validation_augmentation(), 
        preprocessing=get_preprocessing(preprocessing_fn)
    ) 

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
    valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0)
    loss = smp.utils.losses.DiceLoss()
    metrics = [
        smp.utils.metrics.IoU(threshold=0.5),
        DCS()

    ]

    optimizer = torch.optim.Adam([ 
        dict(params=model.parameters(), lr=0.0001),
    ])
    train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
    )
    min_loss = 100000000
    train_history=defaultdict(list)
    valid_history=defaultdict(list)

    for i in range(0, EPOCHS):

        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)

        # do something (save model, change lr, etc.)
        if min_loss > valid_logs['dice_loss']:
            min_loss = valid_logs['dice_loss']
            torch.save(model, os.path.join(WEIGHTS_PATH,f'best_{str(i)}_{round(min_loss,4)}.pt'))
            print('Model saved!')

        if i == 25:
            optimizer.param_groups[0]['lr'] = 1e-5
            print('Decrease decoder learning rate to 1e-5!')
        # Maintain History
        for log_key in train_logs.keys():
            train_history[log_key].append(train_logs[log_key])
            valid_history[log_key].append(valid_logs[log_key])
    pd.DataFrame(valid_history).to_csv(os.path.join(WEIGHTS_PATH,'validation_logs.csv'))
    pd.DataFrame(train_history).to_csv(os.path.join(WEIGHTS_PATH,'train_logs.csv'))
    print(f"{WEIGHTS_PATH} Completed!!!")